* （1）决定一个指标或分数，对电影进行评价；
* （2）计算每一部电影的分数；
* （3）根据分数对电影进行排序，输出top10排行结果。

采用以下评分公式：movie_weight = v/(v+m)*R + m/(v+m)*C
其中，
v是某电影参与评分的观众的个数，
m是筛选的评分观众阈值，即，如果某个电影评分的个数低于阈值，则该电影将被忽略；
R是该电影的评分评分；
C是所有电影的平均分。
m的取值可以根据自己的目的自由选取。
在下面的模型中，采用90分位值，即只选取评论人数为前10%的电影进行分析。

In [2]:
import pandas as pd
import numpy as np

In [3]:
movies = pd.read_csv("dataset/movie_metadata.csv")
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [4]:
movies_mid = movies.loc[:,['movie_title','num_voted_users','imdb_score']]
movies_mid.head()

,movie_title,num_voted_users,imdb_score
0,Avatar,886204,7.9
1,Pirates of the Caribbean: At World's End,471220,7.1
2,Spectre,275868,6.8
3,The Dark Knight Rises,1144337,8.5
4,Star Wars: Episode VII - The Force Awakens ...,8,7.1


In [5]:
# 所有电影平均得分
C = movies_mid['imdb_score'].mean()
print(C)

6.442137616498111


In [6]:
# m为90%处的值
m = movies_mid['num_voted_users'].quantile(0.9)
print(m)

216562.0


In [7]:
# 获取只满足要求电影
q_movies = movies_mid.copy().loc[movies_mid["num_voted_users"] > m]
q_movies.shape

(505, 3)

In [10]:
# 定义一个函数，返回满足要求电影权重得分
def weighted_rating(x,m=m,C=C):
    v = x['num_voted_users']
    R = x['imdb_score']
    return (v / (v+m)*R)+(m/(m+v)*C)

In [11]:
# 计算满足要求权重得分
q_movies['score'] = q_movies.apply(weighted_rating,axis=1)
q_movies = q_movies.sort_values('score',ascending=False)
q_movies.head(10)

,movie_title,num_voted_users,imdb_score,score
1937,The Shawshank Redemption,1689764,9.3,8.975342
3466,The Godfather,1155770,9.2,8.764793
66,The Dark Knight,1676169,9.0,8.707335
3355,Pulp Fiction,1324680,8.9,8.554642
339,The Lord of the Rings: The Return of the King,1215718,8.9,8.528369
97,Inception,1468200,8.8,8.496917
683,Fight Club,1347461,8.8,8.473519
836,Forrest Gump,1251222,8.8,8.452113
2837,The Godfather: Part II,790926,9.0,8.450181
270,The Lord of the Rings: The Fellowship of the R...,1238746,8.8,8.449130
